# 贪心算法 (Greedy Algorithm)

在求解问题时，每一步都做出当前最优的选择（贪心选择），最终可达到该问题的整体最优解。(贪心算法是动态规划的一种特殊情况）

## 1 - 部分背包问题

背包的容量为W kg，有n种不同的食物，其重量分别为[w1, w2, ..., wn], 总价分别为[v1, v2, ..., vn]，求背包所装食物的价值最大为多少？

**策略**：

- 计算各种食物的单价；
- 按单价由高到低将食物装入背包中，直至装满。

In [1]:
def FractionalKnapsack(capacity, w, v):
    n = len(w)
    value_per_weight = [0] * n
    
    # 对物品按单价从小到大排序，储存格式为（value_per_weight, weight)
    for i in range(n):
        value_per_weight[i] = (v[i] / w[i], w[i])
    value_per_weight.sort(reverse=True)
    
    total_value = 0
    i = 0
    while i < n and capacity > 0:
        tmp = min(capacity, value_per_weight[i][1])
        total_value += tmp * value_per_weight[i][0]
        capacity -= tmp
        i += 1
    return total_value

In [2]:
# test case 1
capacity = 50
w = [20, 50, 30]
v = [60, 100, 120]
FractionalKnapsack(capacity, w, v)

180.0

In [3]:
# test case 1
capacity = 50
w = [10, 20, 15]
v = [60, 100, 120]
FractionalKnapsack(capacity, w, v)

280.0

## 2 - 区间覆盖问题

x1,x2,… ,xn是直线上的n个点，用固定长度为k的闭区间去覆盖这n个点，需要至少多少个闭区间才能将这些点全部覆盖?

策略：

- 把这些点按从左到右排序；
- 从最左侧的点开始，划长度为k的闭区间，将此区间覆盖的点从列表中去掉，然后继续用此方法覆盖剩下的点；
- 最后得到的区间总数即为区间最小值，

In [4]:
def PointsCover(x, k):
    x.sort()
    count = 0
    i = 0
    while i < len(x):
        right = x[i] + k
        while i < len(x) and x[i] <= right:
            i += 1
        count += 1
    return count

In [5]:
# test case
x = [1, 5, -2, 7, 3]
k = 3
PointsCover(x, k)

3

### 问题变形

#### 2.1 - 车的最少加油次数

一辆车要从A地开到B地，两地距离为d km。A和B之间共有n个加油站，它们与A的距离分别为x1<=x2<=x3<=…<=xn km。已知该车在满油情况下能开m km，假设车从A地以满油状态出发，则到达B地时至少要加几次油？

策略：车每次都开到能到达的最远的加油站去加油，这样能保证总的加油次数最少。

In [6]:
def MinRefills(d, x, m):
    # x为加油站与A地的距离的列表（由近到远排序）
    n = len(x)
    x += [d, 0] # 在x中加入B地和A地的位置
    count = 0
    i = -1 # 当前站点的编号，起始点为-1，即A地
    while i < n:
        # 如果两个站点的距离>m，则汽车无法到达B地
        if x[i + 1] - x[i] > m:
            return -1
        # 从当前站点出发，汽车能到达的最远处
        right = x[i] + m
        while i < n and x[i + 1] <= right:
            i += 1
        # 如果i=n，则汽车已到达B地，无需再加油
        if i < n:
            count += 1
    return count

In [7]:
# test case 1
d = 950
x = [200, 375, 550, 750]
m = 400
MinRefills(d, x, m)

2

In [8]:
# test case 2
d = 10
x = [1, 2, 5, 9]
m = 3
MinRefills(d, x, m)

-1

In [9]:
# test case 3
d = 200
m = 250
x = [100, 150]
MinRefills(d, x, m)

0

#### 2.2 - 用最少的点去覆盖线段

给定n条线段[[a0, b0], [a1, b1], ..., [an-1, bn-1]], 找到数量最少的m个点，使每条线段都至少含有一个点（即对每一条线段[ai, bi]，都有一个点x满足ai <= x <= bi)。要求输出点的坐标集合。

<img src="images/greedy algorithms/min points.png" style="height:170px;">

In [10]:
def MinPoints(seg):
    seg.sort() # 将线段按起点由小到大排列
    n = len(seg)
    res = []
    i = 0
    while i < n:
        right = seg[i][1] # 能共享一个点的线段的右边界
        # 如果一个线段的起点小于右边界，则它们能公用一个点
        while i < n - 1 and seg[i + 1][0] <= right:
            i += 1
            # 右边界应为所有线段的终点的最小值
            right = min(seg[i][1], right)
        res.append(right)
        i += 1
    return res 

In [11]:
# test case 1
seg = [(1, 3), (2, 5), (3, 6)]
MinPoints(seg)

[3]

In [12]:
# test case 2
seg = [(4, 7), (1, 3), (2, 5), (5, 6)]
MinPoints(seg)

[3, 6]

## 3 - 分糖果

有m个糖果和n个小孩，每个糖果的大小为[s1, s2, ..., sm]， 每个小孩对糖果的需求为[g1, g2, ..., gn]。只有当糖果的大小满足小孩的需求时，小孩才能得到满足。请问至多有几个小孩能得到满足？

策略：

- 把小孩的需求和糖果的大小分别按从小到大排序；
- 优先满足需求最小的小孩，即把剩余糖果中能满足他的最小糖果发给他。

In [13]:
def Candies(sizes, needs):
    sizes.sort()
    needs.sort()
    count = 0
    
    i, j = 0, 0
    while i < len(sizes):
        while i < len(sizes) and sizes[i] < needs[j]:
            i += 1
        if i < len(sizes):
            count += 1
            j += 1
            i += 1
    return count

In [14]:
# test case
s = [6, 1, 20, 3, 8]
g = [5, 10, 2, 9, 15, 9]
Candies(s, g)

3

### 问题变形

#### 3.1 - 奖励糖果

你有n个糖果，你想用这些糖果给一个比赛的前k名作为奖励，要求名次越高得到的糖果越多。请找到能让最多孩子获得奖励的分配方法（即使k值最大）。

策略：倒数第一名奖励1个糖果，倒数第二名奖励2个糖果，……，必须倒数第i+1的糖果比倒数第i名的多，才能给倒数第i名分配i个糖果，否则倒数第i名（也就是第一名）应该获得剩余的所有糖果。

In [15]:
def Prizes(n):
    i = 1
    res = []
    while n > 2 * i:
        res.append(i)
        n -= i
        i += 1
    res.append(n)
    return res

In [16]:
# test case
Prizes(20)

[1, 2, 3, 4, 10]